# Setup EMME / Python Library / Utitlity Functions

In [1]:
import inro.modeller as _m
import inro.emme.desktop as _d
import csv
import os
import multiprocessing
import numpy as np
import pandas as pd
import sqlite3
import datetime
import traceback as _traceback

dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()

util = _m.Modeller().tool("translink.util")

In [10]:
ebs = de.databases()
database_object = {}
for eb in ebs:
    title = eb.title()
    database_object[title] = eb

def open_emmbank(target_title):
    
    # Scenario emmebank open error check
    if not(target_title in database_object.keys()):
        raise Exception(target_title + " Emmebank is not in the project")
    
    # open scenario emmebank
    eb = database_object[target_title]
    eb.open()
    eb = _m.Modeller().emmebank
    return eb

In [11]:
# make sure modeller is closed or it will print to the python console in there
for eb in database_object.keys():
    print eb

UBCx_Ph2_2035_Alt1A_2021-02-12
UBCx_Ph2_2050_Alt1C_2021-02-12
UBCx_Ph2_2035_Alt1D_2021-02-12
UBCx_Ph2_2050_Alt3B_2021-02-12
UBCx_Ph2_2035_Alt3B_2021-02-12
UBCx_Ph2_2035_Alt1C_2021-02-12
UBCx_Ph2_2050_Alt2A_2021-02-12
UBCx_Ph2_2050_Alt2B_2021-02-12
UBCx_Ph2_2050_Alt4C_2021-02-12
UBCx_Ph2_2035_Alt1B_2021-02-12
UBCx_Ph2_2035_Alt4A_2021-02-12
UBCx_Ph2_2035BAU_2021-02-12
UBCx_Ph2_2050BAU_2021-02-12
UBCx_Ph2_2050_Alt4A_2021-02-12
Minimal Base Databank
UBCx_Ph2_2035_Alt2B_2021-02-12
UBCx_Ph2_2050_Alt3C_2021-02-12
UBCx_Ph2_2035_Alt2A_2021-02-12
UBCx_Ph2_2035_Alt3A_2021-02-12
UBCx_Ph2_2050_Alt3A_2021-02-12
UBCx_Ph2_2050_Alt4B_2021-02-12
UBCx_Ph2_2050_Alt1D_2021-02-12
UBCx_Ph2_2050_Alt1A_2021-02-12
UBCx_Ph2_2035_Alt4B_2021-02-12
UBCx_Ph2_2035_Alt4C_2021-02-12
UBCx_Ph2_2050_Alt1B_2021-02-12
UBCx_Ph2_2035_Alt3C_2021-02-12


In [4]:
def df_pivot(table, index, columns, values):
    
    table = pd.pivot(table, index=index,
                          columns=columns,values=values).reset_index().reset_index()
    table.columns = ["index"]+list(table.columns)[1:]
    table = table.drop(columns="index")
    return table

In [5]:
# print table in notebook results
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

# Scenarios and Assumptions Info

In [7]:
scenarios_input = pd.read_csv(os.path.join("..","EconomicAnalysis\EconomicAnalysisTool_Input.csv"))

# load alternative databanks
alternative_table = scenarios_input[["Alternative","Alternative Databank","Horizon"]]
bau_table = pd.DataFrame()
bau_table[["Alternative Databank","Horizon"]] = scenarios_input[["BAU Databank","Horizon"]]
bau_table["Alternative"] = "BAU"
scenarios_table = pd.DataFrame()
scenarios_table[["Scenario","Databank","Horizon"]] = pd.concat([bau_table,alternative_table], 
                                                               sort=False)[["Alternative","Alternative Databank","Horizon"]]
scenarios_table = scenarios_table.reset_index(drop=True).drop_duplicates()
scenarios_table

,Scenario,Databank,Horizon
0,BAU,UBCx_Ph2_2035BAU_2021-02-12,2035
12,BAU,UBCx_Ph2_2050BAU_2021-02-12,2050
24,Alt1A,UBCx_Ph2_2035_Alt1A_2021-02-12,2035
25,Alt1B,UBCx_Ph2_2035_Alt1B_2021-02-12,2035
26,Alt1C,UBCx_Ph2_2035_Alt1C_2021-02-12,2035
27,Alt1D,UBCx_Ph2_2035_Alt1D_2021-02-12,2035
28,Alt2A,UBCx_Ph2_2035_Alt2A_2021-02-12,2035
29,Alt2B,UBCx_Ph2_2035_Alt2B_2021-02-12,2035
30,Alt3A,UBCx_Ph2_2035_Alt3A_2021-02-12,2035
31,Alt3B,UBCx_Ph2_2035_Alt3B_2021-02-12,2035


# A. Customer Service & User Experience

### *) Average Transit Travel Time
AM Transit Travel Time from Arbutus to UBC [min]

In [9]:
fromlocation, fromTAZ = ["Arbutus", 23360]
tolocation, toTAZ = ["UBC1", 21060]

rounding = 0
eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()

fromTAZ_index = ZoneList.index(fromTAZ)
toTAZ_index   = ZoneList.index(toTAZ)

table = ListTable()
table.append(['Scenario \ HorizonYear', '2017', '2035', '2050'])

transit_tt = scenarios_table.copy()
transit_tt["TransitTravelTime"] = -1
for index,row in transit_tt.iterrows():
    eb = open_emmbank(row["Databank"])
    
    # calculate bus travel time
    bus_time  = util.get_matrix_numpy(eb, "AmBusIvtt")
    bus_time += util.get_matrix_numpy(eb, "AmBusWait")
    bus_time += util.get_matrix_numpy(eb, "AmBusAux")
    bus_time += util.get_matrix_numpy(eb, "AmBusBoard")
    # calculate rail travel time
    rail_time  = util.get_matrix_numpy(eb, "AmRailIvtt")
    rail_time += util.get_matrix_numpy(eb, "AmRailIvttBus")
    rail_time += util.get_matrix_numpy(eb, "AmRailWait")
    rail_time += util.get_matrix_numpy(eb, "AmRailAux")
    rail_time += util.get_matrix_numpy(eb, "AmRailBoard")
    # calculate wce travel time
    wce_time  = util.get_matrix_numpy(eb, "AmWceIvtt")
    wce_time += util.get_matrix_numpy(eb, "AmWceIvttRail")
    wce_time += util.get_matrix_numpy(eb, "AmWceIvttBus")
    wce_time += util.get_matrix_numpy(eb, "AmWceWait")
    wce_time += util.get_matrix_numpy(eb, "AmWceAux")
    wce_time += util.get_matrix_numpy(eb, "AmWceBoard")
    # calulate transit travel time = min(bus, rail, wce)
    transit_time = np.minimum(bus_time, rail_time)
    transit_time = np.minimum(transit_time, wce_time)
    
    transit_tt.at[index,"TransitTravelTime"] = transit_time[fromTAZ_index][toTAZ_index]
    
transit_tt = df_pivot(transit_tt, "Scenario", "Horizon", "TransitTravelTime")
transit_tt

,Scenario,2035
0,Alt_1A,21
1,BAU,30


### A1) Reduce Transit Travel Times
Annual System-wide Transit Time Savings in Hours (for BAU/Existing demand)

In [8]:
#re-export transit time savings without perception factors
def ExportData(eb, Dict, filename):
    util = _m.Modeller().tool("translink.util")
        
    OutputPath = os.path.join(util.get_eb_path(_m.Modeller().emmebank), 'EconomicAnalysis')
    if not os.path.exists(OutputPath):
        os.makedirs(OutputPath)
    OutputFile = os.path.join(OutputPath, filename)
    np.savez_compressed(OutputFile, **Dict)

def rename_ROH_TransitTimeCost(eb, mf_number):
    #convert matrix name to npz export name
        
    matrix_name = eb.matrix("mf{}".format(mf_number)).name
    Time_of_Day = matrix_name[:2]
    if "Rail" in matrix_name:
        Class = "RAL"
    else:
        Class = matrix_name[2:5]
    export_name = Time_of_Day + "C" + Class + "9"
    return export_name.upper()

def Export_ROH_TimeCost(eb):
    util = _m.Modeller().tool("translink.util")
    with_previous_export = os.path.isfile(os.path.join(util.get_eb_path(eb), 'EconomicAnalysis', 'ROH_Time_TransitRealTime.npz'))
    if with_previous_export:
        return # do not re-export
    
    Time_Dict = {}
    # list Transit Fare matrix number to be exported
    matrix_list = []
    matrix_list += [5304, 5314, 5324] # AM/MD/PM Bus Fare
    matrix_list += [5505, 5515, 5525] # AM/MD/PM Rail Fare
    matrix_list += [5706, 5726] # AM/PM WCE Fare
    NoTAZ = len(util.get_matrix_numpy(eb, "zoneindex"))
    for mat_id in matrix_list:
        mat_name = "mf{}".format(mat_id)
        name_key = rename_ROH_TransitTimeCost(eb, mat_id)
        
        name_key = name_key[:2] + "M" + name_key[-4:]
        TimeMatrixCount = int(mat_name[-1])
        Time_Dict[name_key] = np.zeros((NoTAZ,NoTAZ))
        for TimeMatrixDigit in range(0,TimeMatrixCount):
            transit_perception_factor = 1.0
            time_mat_name = mat_name[:-1] + str(TimeMatrixDigit)
            Time_Dict[name_key] += util.get_matrix_numpy(eb, time_mat_name)*transit_perception_factor
            
    ExportData(eb, Time_Dict, "ROH_Time_TransitRealTime.npz")
    
for index, row in scenarios_input.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    eb = open_emmbank(ProjectScenarioFolder)
    eb = _m.Modeller().emmebank
    Export_ROH_TimeCost(eb)
    eb = open_emmbank(BAUScenarioFolder)
    eb = _m.Modeller().emmebank
    Export_ROH_TimeCost(eb)

In [13]:
transit_tt_savings = scenarios_input.copy()
transit_tt_savings["transit_time_savings[hours]"]=-1

result = ['Person-Hours by Transit Mode','N/A', 'N/A']
rounding = -2

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [4.24, 4.85, 4.22],
                     "RAL": [4.24, 4.85, 4.22],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":276,"RAL":276,"WCE":224,"LGV":313,"HGV":276}

eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
NoTAZ = len(ZoneList)
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

TAZ_Result = {}
TAZ_Result["TAZ"] = ZoneList
    
for index, row in transit_tt_savings.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis\\ROH_Demand.npz")
    #Base_Time  = np.load(project_dir+BAUScenarioFolder+"/EconomicAnalysis/ROH_Time.npz")
    Base_Time= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis\\ROH_Time_TransitRealTime.npz")
    Altr_Demand= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis\\ROH_Demand.npz")
    #Altr_Time  = np.load(project_dir+ProjectScenarioFolder+"/EconomicAnalysis/ROH_Time.npz")
    Altr_Time= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis\\ROH_Time_TransitRealTime.npz")
    
    Annual_Time_BenefitMinutes = 0
    
    for mode in ["BUS", "RAL"]:
        mode = mode + "9"
        
        Time_Benefit_AM = np.minimum(Base_Demand["AMT"+mode], Altr_Demand["AMT"+mode]) * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode])
        Time_Benefit_MD = np.minimum(Base_Demand["MDT"+mode], Altr_Demand["MDT"+mode]) * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode])
        Time_Benefit_PM = np.minimum(Base_Demand["PMT"+mode], Altr_Demand["PMT"+mode]) * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode])
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result =  int(round((Annual_Time_BenefitMinutes.sum())/60,rounding)) #convert minutes to hours
    transit_tt_savings.at[index,"transit_time_savings[hours]"] = result
    
    scenario_name = row["Alternative"]+"_"+str(row["Horizon"])
    TAZ_Result[scenario_name] = np.sum(Annual_Time_BenefitMinutes,axis=1)

TAZ_Result=pd.DataFrame.from_dict(TAZ_Result,orient='index').transpose()
TAZ_Result_Header = ['TAZ']  + [col for col in TAZ_Result if col != 'TAZ']
TAZ_Result=TAZ_Result[TAZ_Result_Header]
TAZ_Result.to_csv("A1.csv",index=False)
    
transit_tt_savings = df_pivot(transit_tt_savings, "Alternative", "Horizon", "transit_time_savings[hours]")
transit_tt_savings

,Alternative,2035
0,Alt_1A,3039500


### A3) Improve Safety and Security
2035/2050 Reduction in accidents and collisions (vkt-based)

In [12]:
def compute_network_based_vkt(sc):
    util = _m.Modeller().tool("translink.util")
    calc_link = _m.Modeller().tool("inro.emme.network_calculation.network_calculator")
    
    spec = {"result": None, "expression": "(@sov1+@sov2+@sov3+@sov4)*length", "selections": {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    SOV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@hov1+@hov2+@hov3)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    HOV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@lgvol/1.5)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    LGV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "(@hgvol/2.5)*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    HGV_VKT = calc_link(spec)["sum"]
    spec = {"result": None, "expression": "volad*length", "selections":  {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    BUS_VKT = calc_link(spec)["sum"]
    return SOV_VKT, HOV_VKT, LGV_VKT, HGV_VKT, BUS_VKT

rounding = -2
        
TimeofdayToDaily_factors = {"SOV": [3.44, 8.41, 3.95], #AM,MD,PM
                            "HOV": [1.51, 8.58, 5.32],
                            "BUS": [4.24, 4.85, 4.22],
                            "LGV": [3.59, 5.63, 6.17],
                            "HGV": [4.88, 5.43, 6.36]}
DailyToAnnual_factors = [335,335,313,276,276] #SOV,HOV,LGV,HGV,BUS

vkt_table = scenarios_input.copy()
vkt_table["Daily"] = -1.0
vkt_table["Annual"] = -1.0
vkt_table["AM Peak"] = -1.0
vkt_table["PM Peak"] = -1.0

vkt_percapita_table = scenarios_input.copy()
vkt_percapita_table["Daily Reduction per Capita"] = -1.0
vkt_percapita_table["Annual Reduction per Capita"] = -1.0

vkt_table_UBC = scenarios_input.copy()
vkt_table_UBC["Annual VKT Reduction"] = -1.0
vkt_table_UBC["Annual VKT Reduction per Capita"] = -1.0

annual_vkt_reduction_by_mode = scenarios_input.copy()
annual_vkt_reduction_by_mode["Auto"] = -1.0
annual_vkt_reduction_by_mode["Bus"] = -1.0
annual_vkt_reduction_by_mode["LGV"] = -1.0
annual_vkt_reduction_by_mode["HGV"] = -1.0

rounding = -2

for index, row in vkt_table.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    #get Project VKT Dictionary
    eb = open_emmbank(ProjectScenario)
    am_scenario = int(eb.matrix("ms2").data)
    md_scenario = int(eb.matrix("ms3").data)
    pm_scenario = int(eb.matrix("ms4").data)
    Project_VKT = {}
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number in [am_scenario, md_scenario, pm_scenario]:
            de.replace_primary_scenario(sc)
            TimeOfDay_Index = [am_scenario, md_scenario, pm_scenario].index(scenario_number)
            TimeOfDays = ["AM","MD","PM"]
            Project_VKT[TimeOfDays[TimeOfDay_Index]]=compute_network_based_vkt(sc)
    
    #get BAU VKT Dictionary
    eb = open_emmbank(BAUScenario)
    am_scenario = int(eb.matrix("ms2").data)
    md_scenario = int(eb.matrix("ms3").data)
    pm_scenario = int(eb.matrix("ms4").data)
    BAU_VKT = {}
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number in [am_scenario, md_scenario, pm_scenario]:
            de.replace_primary_scenario(sc)
            TimeOfDay_Index = [am_scenario, md_scenario, pm_scenario].index(scenario_number)
            TimeOfDay = ["AM","MD","PM"]
            BAU_VKT[TimeOfDay[TimeOfDay_Index]]=compute_network_based_vkt(sc)
    
    population = util.get_matrix_numpy(eb, "mo10").sum()
    
    #Compute AM/MD/PM/Daily/Annual VKT
    VKT_Summary = []
    for VKT_Dict in [BAU_VKT, Project_VKT]:
        VKT_ScenarioSummary = []
        DailyVKT = 0
        AnnualVKT = 0
        SOV_AnnualVKT = 0
        HOV_AnnualVKT = 0
        LGV_AnnualVKT = 0
        HGV_AnnualVKT = 0 
        BUS_AnnualVKT = 0        
        for TimePeriod in ["AM","MD","PM"]:
            SOV,HOV,LGV,HGV,BUS = VKT_Dict[TimePeriod]
            VKT_ScenarioSummary.append(SOV+HOV+LGV+HGV)
            
            TimeOfDay_Index = ["AM","MD","PM"].index(TimePeriod)
            DailyVKT += SOV*TimeofdayToDaily_factors["SOV"][TimeOfDay_Index]
            DailyVKT += HOV*TimeofdayToDaily_factors["HOV"][TimeOfDay_Index]
            DailyVKT += LGV*TimeofdayToDaily_factors["LGV"][TimeOfDay_Index]
            DailyVKT += HGV*TimeofdayToDaily_factors["HGV"][TimeOfDay_Index]
            
            SOV_AnnualVKT += SOV*TimeofdayToDaily_factors["SOV"][TimeOfDay_Index]*DailyToAnnual_factors[0]
            HOV_AnnualVKT += HOV*TimeofdayToDaily_factors["HOV"][TimeOfDay_Index]*DailyToAnnual_factors[1]
            LGV_AnnualVKT += LGV*TimeofdayToDaily_factors["LGV"][TimeOfDay_Index]*DailyToAnnual_factors[2]
            HGV_AnnualVKT += HGV*TimeofdayToDaily_factors["HGV"][TimeOfDay_Index]*DailyToAnnual_factors[3]
            
            BUS_AnnualVKT += BUS*TimeofdayToDaily_factors["BUS"][TimeOfDay_Index]*DailyToAnnual_factors[4]
            
        AnnualVKT += SOV_AnnualVKT
        AnnualVKT += HOV_AnnualVKT
        AnnualVKT += LGV_AnnualVKT
        AnnualVKT += HGV_AnnualVKT
            
        VKT_ScenarioSummary.append(DailyVKT)
        VKT_ScenarioSummary.append(AnnualVKT)
        VKT_ScenarioSummary.append(SOV_AnnualVKT)
        VKT_ScenarioSummary.append(HOV_AnnualVKT)
        VKT_ScenarioSummary.append(LGV_AnnualVKT)
        VKT_ScenarioSummary.append(HGV_AnnualVKT)
        VKT_ScenarioSummary.append(BUS_AnnualVKT)
        VKT_Summary.append(VKT_ScenarioSummary)
            
    VKT_Reduction = [BAU_VKT_i - Project_VKT_i for BAU_VKT_i, Project_VKT_i in zip (VKT_Summary[0], VKT_Summary[1])]
    VKT_Reduction_percapita = [round(x/population, 2) for x in VKT_Reduction]
    VKT_Reduction_round = [int(round(x, rounding)) for x in VKT_Reduction]
    
    vkt_table.at[index,"Daily"] = VKT_Reduction_round[3]
    vkt_table.at[index,"Annual"] = VKT_Reduction_round[4]
    vkt_table.at[index,"AM Peak"] = VKT_Reduction_round[0]
    vkt_table.at[index,"PM Peak"] = VKT_Reduction_round[2]
    
    vkt_percapita_table.at[index,"Daily Reduction per Capita"] = VKT_Reduction_percapita[3]
    vkt_percapita_table.at[index,"Annual Reduction per Capita"] = VKT_Reduction_percapita[4]
    
    vkt_table_UBC.at[index,"Annual VKT Reduction"] = VKT_Reduction_round[4]
    vkt_table_UBC.at[index,"Annual VKT Reduction per Capita"] = VKT_Reduction_percapita[4]
    
    annual_vkt_reduction_by_mode.at[index,"Auto"] = VKT_Reduction[5] + VKT_Reduction[6]
    annual_vkt_reduction_by_mode.at[index,"Bus"]  = VKT_Reduction[9]
    annual_vkt_reduction_by_mode.at[index,"LGV"]  = VKT_Reduction[7]
    annual_vkt_reduction_by_mode.at[index,"HGV"]  = VKT_Reduction[8]
    
vkt_table = vkt_table.drop(columns=["Alternative Databank","BAU Databank"])
vkt_percapita_table = vkt_percapita_table.drop(columns=["Alternative Databank","BAU Databank"])
vkt_table_UBC = vkt_table_UBC.drop(columns=["Alternative Databank","BAU Databank"])
annual_vkt_reduction_by_mode = annual_vkt_reduction_by_mode.drop(columns=["Alternative Databank","BAU Databank"])

In [13]:
collision_factor = 0.65/1000000

collision_table = vkt_table.copy()
collision_table["Annual Collision Reduction"] = collision_table["Annual"]*collision_factor
collision_table = collision_table.drop(columns=["Daily","Annual","AM Peak","PM Peak"])
collision_table.to_csv("A3.csv",index=False)
collision_table

,Alternative,Horizon,Annual Collision Reduction
0,Alt1A,2035,37.935365
1,Alt1B,2035,40.320735
2,Alt1C,2035,41.900625
3,Alt1D,2035,44.128890
4,Alt2A,2035,39.993135
5,Alt2B,2035,44.225740
6,Alt3A,2035,33.462520
7,Alt3B,2035,35.639370
8,Alt3C,2035,40.237990
9,Alt4A,2035,32.289595


### A4a) Baseline Auto Travel Time Savings
Annual Auto Travel Time Savings in Hours

In [18]:
auto_tt_savings = scenarios_input.copy()
auto_tt_savings["auto_time_savings[hours]"]=-1

result = ['Person-Hours by Auto Mode','N/A', 'N/A']
rounding = -2
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [4.24, 4.85, 4.22],
                     "RAL": [4.24, 4.85, 4.22],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":276,"RAL":276,"WCE":224,"LGV":313,"HGV":276}

TAZ_Result_auto = {}
TAZ_Result_auto["TAZ"] = ZoneList

for index, row in auto_tt_savings.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Base_Time  = np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    Altr_Demand= np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Altr_Time  = np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    
    Annual_Time_BenefitMinutes = 0
    
    for mode in mode_list["Auto"]:
        
        Time_Benefit_AM = (np.minimum(Base_Demand["AMT"+mode], Altr_Demand["AMT"+mode]) * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode]))
        Time_Benefit_MD = (np.minimum(Base_Demand["MDT"+mode], Altr_Demand["MDT"+mode]) * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode]))
        Time_Benefit_PM = (np.minimum(Base_Demand["PMT"+mode], Altr_Demand["PMT"+mode]) * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode]))
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result=int(round((Annual_Time_BenefitMinutes.sum())/60,rounding))#convert minutes to hours
    auto_tt_savings.at[index,"auto_time_savings[hours]"] = result
    
    scenario_name = row["Alternative"]+"_"+str(row["Horizon"])
    TAZ_Result_auto[scenario_name] = np.sum(Annual_Time_BenefitMinutes,axis=1)

TAZ_Result_auto=pd.DataFrame.from_dict(TAZ_Result_auto,orient='index').transpose()
TAZ_Result_auto_Header = ['TAZ']  + [col for col in TAZ_Result_auto if col != 'TAZ']
TAZ_Result_auto=TAZ_Result_auto[TAZ_Result_auto_Header]
TAZ_Result_auto.to_csv("A4a.csv",index=False)

auto_tt_savings = df_pivot(auto_tt_savings, "Alternative", "Horizon", "auto_time_savings[hours]")
auto_tt_savings

,Alternative,2035
0,Alt_1A,1365100


### A4b) Reduce Road Congestion
Reduction in auto delay along the corridor (Broadway from Commerical Drive to UBC) <br>
AM and PM (both directions) - Link Based Travel Time

In [ ]:
def tag_broadway(eb, scenario):
    util = _m.Modeller().tool("translink.util")
    calc_link = _m.Modeller().tool("inro.emme.network_calculation.network_calculator")
    create_extra = _m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
    tag_tool = _m.Modeller().tool("translink.RTM3Analytics.GeographicTagging")
            
    # tag Broadway corridor
    create_extra(extra_attribute_type="LINK",
                 extra_attribute_name="@broadway",
                 extra_attribute_description="Broadway Corridor",
                 overwrite=True)
    # tag broadwau links
    linkattributeName = "@broadway"
    polygonfile = os.path.abspath(os.path.join(eb.path, "..", "..", 
                                  "Media", "MAE", "Broadway.shp"))
    algorithm = "do_not_change"
    excludeconnector = True
    default = 0
    polygonfield = "broadway"
    scen = eb.scenario(scenario)
    tag_tool(scen,linkattributeName,polygonfile,polygonfield,
             algorithm,excludeconnector,default)
    
    # exclude HOV lanes, mode d = SOV
    spec = {"result":"@broadway", "expression": "@broadway+1", "selections": {"link": "modes=d"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    calc_link(spec)
    spec = {"result":"@broadway", "expression": "(@broadway-1).max.0", "selections": {"link": "all"}, "aggregation": None, "type": "NETWORK_CALCULATION"}
    calc_link(spec)
            
    # save transit segment volume @voltravg
    export_filepath = os.path.join(os.getcwd(), "Broadway.csv")
    link_table.export(export_filepath)

    # append horizon and scenario
    broadyway_traveltime = pd.read_csv(export_filepath)

    # filter the WB routes
    net = scen.get_network()
    for i,row_i in broadyway_traveltime.iterrows():
        broadyway_traveltime.at[i,"i_x"] = net.node(row_i["i"]).x
        broadyway_traveltime.at[i,"j_x"] = net.node(row_i["j"]).x
#     print(broadyway_traveltime.head())
    broadyway_traveltime["dir"] = np.where(broadyway_traveltime.i_x>broadyway_traveltime.j_x, "WB", "EB")
    os.remove(export_filepath)
    
    broadyway_traveltime = broadyway_traveltime[["timau","dir"]].groupby("dir")["timau"].sum().reset_index()
    return broadyway_traveltime
    
root_worksheet_folder = dt.root_worksheet_folder()
link_table_path = root_worksheet_folder.find_item(["MAE","BroadwayTravelTime"])
link_table = link_table_path.open()
link_table.par("ExportColumnSeparator").set(",")

BroadwayTravelTimeSavingResults = scenarios_input.copy()

broadyway_traveltime_result = []
for index, row in BroadwayTravelTimeSavingResults.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    BAU_Alter_Table = []
    for Alternative, databank_name in [["BAU",row["BAU Databank"]],
                                       ["Altr",row["Alternative Databank"]]]:
        # get travel times
        eb = open_emmbank(databank_name)
        am_scenario = int(eb.matrix("ms2").data)
        pm_scenario = int(eb.matrix("ms4").data)
        db = de.active_database()
        AMPM_TravelTime = []
        for sc in db.scenarios():
            scenario_number = sc.number()
            if scenario_number in [am_scenario, pm_scenario]:
                de.replace_primary_scenario(sc)
                TimeOfDay_Index = [am_scenario, pm_scenario].index(scenario_number)

                # tag Broadway corridor
                broadyway_traveltime = tag_broadway(eb, scenario_number)
                broadyway_traveltime["Horizon"] = row["Horizon"]
                broadyway_traveltime["Alternative"] = row["Alternative"]
                broadyway_traveltime["Peak"] = ["AM","PM"][TimeOfDay_Index]
                broadyway_traveltime.rename(columns={"timau":Alternative+"_TravelTime"}, inplace=True)
                AMPM_TravelTime.append(broadyway_traveltime)
        BAU_Alter_Table.append(pd.concat(AMPM_TravelTime).reset_index(drop=True))
        
    broadyway_traveltime = BAU_Alter_Table[0].merge(BAU_Alter_Table[1], how='outer', on=["Horizon","Peak","dir","Alternative"])
                
    broadyway_traveltime = broadyway_traveltime[["Horizon","Peak","dir","Alternative","Altr_TravelTime","BAU_TravelTime"]]
    broadyway_traveltime_result.append(broadyway_traveltime)

link_table.close()
broadyway_traveltime_result = pd.concat(broadyway_traveltime_result).reset_index(drop=True)
broadyway_traveltime_result["Reduction_In_Auto_Delay[min]"] = broadyway_traveltime_result["BAU_TravelTime"]-broadyway_traveltime_result["Altr_TravelTime"]
broadyway_traveltime_result.to_csv("A4b.csv",index=False)
broadyway_traveltime_result

### *) Reduction in Auto Delay (for each trip)
Reduction in congestion minutes from congestion minutes OD matrix <br>
AM SOV3

In [23]:
def run_congestedassignment(eb):
    eb = open_emmbank(eb)
    congestion_assignment = _m.Modeller().tool("translink.congestedassignment")
    am_scen = eb.scenario(int(eb.matrix("msAmScen").data))
    md_scen = eb.scenario(int(eb.matrix("msMdScen").data))
    pm_scen = eb.scenario(int(eb.matrix("msPmScen").data))
    congestion_assignment(eb, am_scen, md_scen, pm_scen, "timau", 1)
    
for index,row in scenarios_table.iterrows():
    run_congestedassignment(row["Databank"])


In [ ]:
delay_reduction_table = []
header = ["alternative","horizon","from","to","auto delay reduction [min]"]

fromTAZ_List = [["Fleetwood", 63520], 
                ["Langley",   67080]]
toTAZ_List = [["SurreyCentral",       61440], 
              ["CommercialBroadway",  25420], 
              ["ProductionWay",       27520], 
              ["SFU",                 27160], 
              ["CoquitlamCityCentre", 32030]]
eb = _m.Modeller().emmebank
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

table = ListTable()
for index, row in scenarios_input.iterrows():
    ScenarioFolder = row["Alternative Databank"]
    BAUFolder = row["BAU Databank"]
    
    Delay_Project  = np.load(project_dir+"\\"+ScenarioFolder+"\\EconomicAnalysis\\TIMECONG.npz")
    Delay_BAU  = np.load(project_dir+"\\"+BAUFolder+"\\EconomicAnalysis\\TIMECONG.npz")
            
    # each tolocation is a table row
    for tolocation, toTAZ in toTAZ_List:
        toTAZindex = ZoneList.index(toTAZ)
        for fromlocation, fromTAZ in fromTAZ_List:
            fromTAZindex = ZoneList.index(fromTAZ)
            
            #get reliability Table
            CongestionMinutes_Project = Delay_Project["AMSOVTIMECONGVOT3"][fromTAZindex,toTAZindex]
            CongestionMinutes_BAU = Delay_BAU["AMSOVTIMECONGVOT3"][fromTAZindex,toTAZindex]
            delay_reduction_table.append([str(row["Alternative"]), int(row["Horizon"]),
                                              fromlocation, tolocation,
                                              CongestionMinutes_BAU-CongestionMinutes_Project])            

delay_reduction_table = pd.DataFrame(delay_reduction_table, columns=header)

# B. Transportation

### B1) New Daily Transit Trips
Change in System-wide Daily Trips: Transit 
Total Transit Trips Project - BAU

In [15]:
DailyTripsbyMode = _m.Modeller().tool("translink.MAE_DailyTripsByMode")

rounding = -2

import sys #suppress print statements in daily mode share script
sys.stdout = open(os.devnull, 'w')

new_transit_trips = scenarios_input.copy()
new_transit_trips["net new transit trips"] = -1
for index, row in new_transit_trips.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    eb = open_emmbank(ProjectScenario)
    trip_total, trips_by_mode_Dict = DailyTripsbyMode.getDemand(eb)
    ProjectScenario_TransitTrips = trips_by_mode_Dict["Transit"]
    
    eb = open_emmbank(BAUScenario)
    trip_total, trips_by_mode_Dict = DailyTripsbyMode.getDemand(eb)
    BAUScenario_TransitTrips = trips_by_mode_Dict["Transit"]
    
    result = int(round(ProjectScenario_TransitTrips-BAUScenario_TransitTrips,rounding))
    new_transit_trips.at[index,"net new transit trips"] = result
        
sys.stdout = sys.__stdout__ # change print setting to default

new_transit_trips = df_pivot(new_transit_trips, "Alternative", "Horizon", "net new transit trips")
new_transit_trips.to_csv("B1.csv",index=False)    
new_transit_trips

,Alternative,2035,2050
0,Alt1A,19300,23200
1,Alt1B,20900,24700
2,Alt1C,22400,26900
3,Alt1D,23900,28400
4,Alt2A,21400,25800
5,Alt2B,24600,29700
6,Alt3A,17800,22100
7,Alt3B,19500,23700
8,Alt3C,23300,28500
9,Alt4A,17700,22000


### B2) Increase Sustainable Mode Share
Daily trips by mode, by origin and by destination

In [17]:
DailyTripsbyMode = _m.Modeller().tool("translink.MAE_DailyTripsByMode")

TAZ_Result_TripByMode = {}
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
TAZ_Result_TripByMode["TAZ"] = ZoneList

import sys #suppress print statements in daily mode share script
sys.stdout = open(os.devnull, 'w')

for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    scenario_label = row["Scenario"]+"_"+str(row["Horizon"])
    
    TripsByMode = DailyTripsbyMode.getDemand(eb,export_vector=True)
    for key, value in TripsByMode.iteritems():
        TAZ_Result_TripByMode[scenario_label+"_"+key]=value

TAZ_Result_TripByMode=pd.DataFrame.from_dict(TAZ_Result_TripByMode,orient='index').transpose()
TAZ_Result_TripByMode_Header = ['TAZ']  + [col for col in TAZ_Result_TripByMode if col != 'TAZ']
TAZ_Result_TripByMode=TAZ_Result_TripByMode[TAZ_Result_TripByMode_Header]
TAZ_Result_TripByMode.to_csv("B2.csv",index=False)    
        
sys.stdout = sys.__stdout__ # change print setting to default

### B3) Increase Capacity to meet future needs
crowding on parallel routes (UBC Screenline) - AM WB

In [ ]:
# transit volume on UBC Screenline
create_extra = _m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
tag_tool = _m.Modeller().tool("translink.RTM3Analytics.GeographicTagging")
root_worksheet_folder = dt.root_worksheet_folder()
link_table_path = root_worksheet_folder.find_item(["MAE","TransitVolume_UBCScreenline"])
link_table = link_table_path.open()
link_table.par("ExportColumnSeparator").set(",")

transit_sl_tt = []
for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    
    # open the scenario
    am_scenario = int(eb.matrix("ms2").data)
    db = de.active_database()
    for sc in db.scenarios():
        scenario_number = sc.number()
        if scenario_number==am_scenario:
            de.replace_primary_scenario(sc)
    
            # tag UBC screenline
            create_extra(extra_attribute_type="LINK",
                         extra_attribute_name="@ubcscreenline",
                         extra_attribute_description="UBC Screenline",
                         overwrite=True)
            # tag UBC screenline links
            linkattributeName = "@ubcscreenline"
            polygonfile = os.path.abspath(os.path.join(eb.path, "..", "..", 
                                          "Media", "MAE", "UBC_Screenlline.shp"))
            algorithm = "tag_max"
            excludeconnector = True
            default = 0
            polygonfield = "SL_NUM"
            scen = eb.scenario(am_scenario)
            tag_tool(scen,linkattributeName,polygonfile,polygonfield,
                     algorithm,excludeconnector,default)
            
            # save transit segment volume @voltravg
            export_filepath = row["Scenario"] + "_" + str(row["Horizon"]) + ".csv"
            export_filepath = os.path.join(os.getcwd(), export_filepath)
            link_table.export(export_filepath)
            
            # append horizon and scenario
            transitvolume_UBC = pd.read_csv(export_filepath)
            transitvolume_UBC["Horizon"] = row["Horizon"]
            transitvolume_UBC["Scenario"] = row["Scenario"]
            
            # filter the WB routes
            net = scen.get_network()
            for i,row_i in transitvolume_UBC.iterrows():
                transitvolume_UBC.at[i,"i_x"] = net.node(row_i["i"]).x
                transitvolume_UBC.at[i,"j_x"] = net.node(row_i["j"]).x
            
            # keep WB routes (ix>jx), drop EB routes
            transitvolume_UBC = transitvolume_UBC[transitvolume_UBC.i_x>transitvolume_UBC.j_x]
            
            transit_sl_tt.append(transitvolume_UBC)
            os.remove(export_filepath)

link_table.close()
transit_sl_tt = pd.concat(transit_sl_tt)[["Horizon","Scenario","Route","Volume","Capacity"]].reset_index()
transit_sl_tt["Scenario"] = transit_sl_tt["Horizon"].apply(str)+"_"+transit_sl_tt["Scenario"]
transit_sl_tt_volume = df_pivot(transit_sl_tt, "Scenario","Route","Volume").fillna(0)
transit_sl_tt_volume["Total_WB"] = transit_sl_tt_volume.iloc[:, 1:].sum(axis=1)
transit_sl_tt_volume["DataType"] = "Volume"
transit_sl_tt_capacity = df_pivot(transit_sl_tt, "Scenario","Route","Capacity").fillna(0)
transit_sl_tt_capacity["Total_WB"] = transit_sl_tt_capacity.iloc[:, 1:].sum(axis=1)
transit_sl_tt_capacity["DataType"] = "Capacity"
transit_screenline = pd.concat([transit_sl_tt_volume, transit_sl_tt_capacity]).reset_index(drop=True)
# rearrange dataframe columns
transit_screenline_header = ['Scenario','DataType'] + [col for col in transit_screenline if (not(col in ["Scenario","DataType"]))]
transit_screenline = transit_screenline[transit_screenline_header]
transit_screenline.to_csv("B3.csv",index=False)    
transit_screenline

### B4) Connect people with places
From Univeristy and Main Mall, transit travel time from all other zones.  AM inbound direction

In [21]:
def get_transit_travel_time(eb):
    # get AM Transit Travel Time Matrix take minimum of Bus/Rail/WCE Travel Time
    tempmatrix = "mf9999"
    util.initmat(eb, tempmatrix, "tempMatrix", "temp matrix", 0) #for matrix data
    specs = []
    specs.append(util.matrix_spec(tempmatrix, "(mf5300+mf5301+mf5302+mf5303).min.(mf5500+mf5501+mf5502+mf5503+mf5504)"))
    specs.append(util.matrix_spec(tempmatrix, tempmatrix+".min.(mf5700+mf5701+mf5702+mf5703+mf5704+mf5705)"))
    util.compute_matrix(specs)
    TransitTravelTime_matrix = util.get_matrix_numpy(eb, tempmatrix)
    util.delmat(eb, tempmatrix)
    return TransitTravelTime_matrix

TransitTravelTime_Table = {}
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
TransitTravelTime_Table["TAZ"] = ZoneList
zone_index = ZoneList.index(21060) # University and Mall at TAZ 21060

for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    transit_traveltime = get_transit_travel_time(eb)
    
    scenario_label = row["Scenario"]+"_"+str(row["Horizon"])
    TransitTravelTime_Table[scenario_label] = transit_traveltime[:,zone_index] # for trips destinated at TAZ

TransitTravelTime_Table=pd.DataFrame.from_dict(TransitTravelTime_Table,orient='index').transpose()
TAZ_Result_TripByMode_Header = ['TAZ']  + [col for col in TransitTravelTime_Table if col != 'TAZ']
TransitTravelTime_Table=TransitTravelTime_Table[TAZ_Result_TripByMode_Header]
TransitTravelTime_Table.to_csv("B4.csv",index=False)        

### *) Number of Trips by Active Modes
1) Incremental Trips by Active Modes to Access Transit against BAU <br>
2) Additional Kilometers Travelled by Active Modes to Access Transit

In [ ]:
def getWalkTripsDistance(eb):
    eb = open_emmbank(eb)
    
    walkTrips = 0
    walkDistance = 0
    walkSpeed = 4.8 #km/hr - speed factor of walking mode
    
    matrix_list =  [["mf314","mf5302", "mf5303", "AM", "BUS"], #AM Bus
                    ["mf315","mf5503", "mf5504", "AM", "RAL"], #AM Rail
                    ["mf316","mf5704", "mf5705", "AM", "WCE"], #AM WCE
                    ["mf334","mf5312", "mf5313", "MD", "BUS"], #MD Bus
                    ["mf335","mf5513", "mf5514", "MD", "RAL"], #MD Rail
                    ["mf354","mf5322", "mf5323", "PM", "BUS"], #PM Bus
                    ["mf355","mf5523", "mf5524", "PM", "RAL"], #PM Rail
                    ["mf356","mf5724", "mf5725", "PM", "WCE"]] #PM WCE

    expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                         "HOV": [1.51, 8.58, 5.32],
                         "BUS": [4.24, 4.85, 4.22],
                         "RAL": [4.24, 4.85, 4.22],
                         "WCE": [3.34,    0, 2.02],
                         "LGV": [3.59, 5.63, 6.17],
                         "HGV": [4.88, 5.43, 6.36]}
    DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":276,"RAL":276,"WCE":224,"LGV":313,"HGV":276}
    
    for trips,auxillaryTime,boardings,TOD,Mode in matrix_list:
        TOD_Index = ["AM","MD","PM"].index(TOD)
        expansion = DailyToAnnual_factors[Mode] * expansion_factors[Mode][TOD_Index]
        
        mf_trips = util.get_matrix_numpy(eb, trips, reshape=False)
        mf_auxillaryTime = util.get_matrix_numpy(eb, auxillaryTime, reshape=False)
        mf_boardings = util.get_matrix_numpy(eb, boardings, reshape=False)
        
        walkTrips += mf_trips * (mf_boardings + 1) * expansion
        walkDistance += mf_trips * (walkSpeed * mf_auxillaryTime/60) * expansion #convert walk time from minutes to hours
    return walkTrips.sum(),walkDistance.sum()


active_trips = scenarios_input.copy()
active_trips["incremental active trips"] = -1
active_trips["additional active vkt"] = -1

rounding = -2

for index, row in active_trips.iterrows():
    ProjectScenario = row["Alternative Databank"]
    BAUScenario = row["BAU Databank"]
    
    WalkTrips_Project, WalkDistance_Project = getWalkTripsDistance(ProjectScenario)
    WalkTrips_BAU,     WalkDistance_BAU     = getWalkTripsDistance(BAUScenario)
    
    incremental_trips = int(round(WalkTrips_Project - WalkTrips_BAU,rounding))
    incremental_vkt = int(round(WalkDistance_Project - WalkDistance_BAU,rounding))
    
    active_trips.at[index,"incremental active trips"] = incremental_trips
    active_trips.at[index,"additional active vkt"] = incremental_vkt
    
active_trips = active_trips.drop(columns=["Alternative Databank","BAU Databank"])
active_trips

# C. Social, Community, Environment

### C1a) System-wide VKT Reduction and VKT Reduction Per Capita

In [19]:
vkt_table_UBC.to_csv("C1a.csv",index=False)  
vkt_table_UBC

,Alternative,Horizon,Annual VKT Reduction,Annual VKT Reduction per Capita
0,Alt1A,2035,58362100.0,15.68
1,Alt1B,2035,62031900.0,16.67
2,Alt1C,2035,64462500.0,17.32
3,Alt1D,2035,67890600.0,18.24
4,Alt2A,2035,61527900.0,16.53
5,Alt2B,2035,68039600.0,18.28
6,Alt3A,2035,51480800.0,13.83
7,Alt3B,2035,54829800.0,14.73
8,Alt3C,2035,61904600.0,16.64
9,Alt4A,2035,49676300.0,13.35


### C1b) Total expected reduction in GHG emissions (tonnes of CO2) and CACs
2035 and 2050

In [20]:
# 2035 Horizon: 2035 BAU Factors in "Onroad EF T2050.xlsx"
# 2050 Horizon: 2035 EV Agg Factors in "Onroad EF T2050.xlsx"
# Factors {mode:[2035,2050]} 
GHG_factor = {"Auto":[ 152,   91], #Co2e (g/VKT)
              "Bus": [ 748,  748],
              "LGV": [ 238,  203],
              "HGV": [1050, 1050]}
CAC_factor = {"Auto":[0.21, 0.21], #CACs (g/VKT)
              "Bus": [0.77, 0.77],
              "LGV": [0.20, 0.18],
              "HGV": [1.78, 1.78]}

emission_table = annual_vkt_reduction_by_mode.copy()
# emission_table["Total"] = emission_table["Auto"] + emission_table["LGV"] + emission_table["HGV"] 

emission_table["GHGs[tonnes]"] = 0
emission_table["CACs[tonnes]"] = 0

for i in range(2):
    horizon = [2035,2050][i]
    #for mode in ["Auto","Bus","LGV","HGV"]:
    for mode in ["Auto","LGV","HGV"]: # do not include bus emissions
        emission_table["GHGs[tonnes]"] = np.where(emission_table["Horizon"]==horizon,
                                                  emission_table["GHGs[tonnes]"]+emission_table[mode]*GHG_factor[mode][i]/1000000,
                                                  emission_table["GHGs[tonnes]"])
        emission_table["CACs[tonnes]"] = np.where(emission_table["Horizon"]==horizon,
                                                  emission_table["CACs[tonnes]"]+emission_table[mode]*CAC_factor[mode][i]/1000000,
                                                  emission_table["CACs[tonnes]"])
emission_table.drop(columns=["Auto","Bus","LGV","HGV"], inplace=True)
emission_table.to_csv("C1b.csv",index=False)  
emission_table

,Alternative,Horizon,GHGs[tonnes],CACs[tonnes]
0,Alt1A,2035,8817.237569,12.205944
1,Alt1B,2035,9378.791760,12.978680
2,Alt1C,2035,9795.254592,13.567142
3,Alt1D,2035,10292.582008,14.243284
4,Alt2A,2035,9334.957803,12.915867
5,Alt2B,2035,10333.825541,14.311911
6,Alt3A,2035,7784.635262,10.769657
7,Alt3B,2035,8277.663732,11.452654
8,Alt3C,2035,9378.639734,12.988043
9,Alt4A,2035,7484.546980,10.353347


### C3 Support Social Equity and Distribution of Benefits
TAZ vector with jobs within 45 and 60 minutes of transit (computed from the full matrix) <br>
Also need the output of the Socioeconomic segmentation model with count of households by income category

In [22]:
Assessibility_Table = {}
ZoneList = util.get_matrix_numpy(eb, "mozoneindex", reshape=False).astype(int).tolist()
Assessibility_Table["TAZ"] = ZoneList
zone_index = ZoneList.index(21060) # University and Mall at TAZ 21060
jobs = util.get_matrix_numpy(eb, "moTotEmp", reshape=False)

for index,row in scenarios_table.iterrows():
    eb = open_emmbank(row["Databank"])
    transit_traveltime = get_transit_travel_time(eb)
    jobs_in_45min = np.where(transit_traveltime<=45, jobs, 0)
    jobs_in_60min = np.where(transit_traveltime<=60, jobs, 0)
    
    scenario_label = row["Scenario"]+"_"+str(row["Horizon"])
    
    Assessibility_Table[scenario_label+"_45min"] = np.sum(jobs_in_45min, axis=1)
    Assessibility_Table[scenario_label+"_60min"] = np.sum(jobs_in_60min, axis=1)

Assessibility_Table=pd.DataFrame.from_dict(Assessibility_Table,orient='index').transpose()
Assessibility_Table_Header = ['TAZ']  + [col for col in Assessibility_Table if col != 'TAZ']
Assessibility_Table=Assessibility_Table[Assessibility_Table_Header]
Assessibility_Table.to_csv("C3.csv",index=False)        

In [23]:
# export the household count by income category for 2035/2050 BAU databanks
df_hh_list = []
for index,row in scenarios_table.iterrows():
    if row["Scenario"]!="BAU":
        continue # to next databank, do not export 
    eb = open_emmbank(row["Databank"])
    
    conn_rtm = util.get_rtm_db(eb)
    df_hh = pd.read_sql("select * from segmentedHouseholds", conn_rtm)
    conn_rtm.close()
    df_hh = df_hh[["TAZ1741","HHInc","CountHHs"]].groupby(["TAZ1741","HHInc"])["CountHHs"].sum().reset_index()
    df_hh = df_pivot(df_hh, "TAZ1741","HHInc","CountHHs").fillna(0)
    df_hh.rename(columns={1: scenario_label+"_LowIncomeHHCount", 
                          2: scenario_label+"_MedIncomeHHCount", 
                          3: scenario_label+"_HigIncomeHHCount"}, inplace=True)
    df_hh_list.append(df_hh)
    
# df_hh_list[0].merge(df_hh_list[1], how='outer', on='TAZ1741')
df_hh_list[0].to_csv("C3_HH.csv",index=False)        

# D. Economic Development

### D3) Reduce Truck Travel Time
1) Daily Truck Travel Time Saving (Hours) <br>
2) Annual Truck Travel Time Saving in dollars

In [16]:
truck_table = scenarios_input.copy()
truck_table["Daily Truck Time Saving Hours"] = -1
truck_table["Annual Truck Time Saving Hours"] = -1
truck_table["Annual Truck Time Savings [2019$]"] = -1

rounding1 = -1
rounding2 = -2
project_dir = os.path.abspath(os.path.join(eb.path,"../.."))

# mode_list = {mode_category: mode_group [mode, mode, mode]}
mode_list = {"Auto": ["SOV1", "SOV2", "SOV3", "SOV4", "HOV1", "HOV2", "HOV3"],
             "Transit": ["BUS", "RAL", "WCE"],
             "Light_Truck": ["LGV"], 
             "Heavy_Truck": ["HGV"]}

expansion_factors = {"SOV": [3.44, 8.41, 3.95],
                     "HOV": [1.51, 8.58, 5.32],
                     "BUS": [4.24, 4.85, 4.22],
                     "RAL": [4.24, 4.85, 4.22],
                     "WCE": [3.34,    0, 2.02],
                     "LGV": [3.59, 5.63, 6.17],
                     "HGV": [4.88, 5.43, 6.36]}

DailyToAnnual_factors = {"SOV":335,"HOV":335,"BUS":276,"RAL":276,"WCE":224,"LGV":313,"HGV":276}

for index, row in truck_table.iterrows():
    ProjectScenarioFolder = row["Alternative Databank"]
    BAUScenarioFolder = row["BAU Databank"]
    
    Base_Demand= np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Demand.npz")
    Base_Time  = np.load(project_dir+"\\"+BAUScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    Altr_Time  = np.load(project_dir+"\\"+ProjectScenarioFolder+"\\EconomicAnalysis/ROH_Time.npz")
    
    Time_Benefit_AM = 0
    Time_Benefit_MD = 0
    Time_Benefit_PM = 0
    Daily_Time_BenefitMinutes = 0
    Annual_Time_BenefitMinutes = 0
    
    for mode in ["LGV","HGV"]:
        mode = mode + "9"
        Time_Benefit_AM = (Base_Demand["AMT"+mode] * (Base_Time["AMM"+mode] - Altr_Time["AMM"+mode])).sum()
        Time_Benefit_MD = (Base_Demand["MDT"+mode] * (Base_Time["MDM"+mode] - Altr_Time["MDM"+mode])).sum()
        Time_Benefit_PM = (Base_Demand["PMT"+mode] * (Base_Time["PMM"+mode] - Altr_Time["PMM"+mode])).sum()
        
        AM_Fac, MD_Fac, PM_Fac = expansion_factors[mode[:3]]
        Daily_Time_BenefitMinutes += Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac
        
        AnnualFactor = DailyToAnnual_factors[mode[:3]]
        Annual_Time_BenefitMinutes += (Time_Benefit_AM * AM_Fac + Time_Benefit_MD * MD_Fac + Time_Benefit_PM * PM_Fac)*AnnualFactor
    
    result = int(round((Daily_Time_BenefitMinutes)/60,rounding1))#convert minutes to hours
    truck_table.at[index,"Daily Truck Time Saving Hours"] = result
    result = int(round((Annual_Time_BenefitMinutes)/60,rounding1))#convert minutes to hours
    truck_table.at[index,"Annual Truck Time Saving Hours"] = result
    
    TruckValueOfTime = 31.25 # 2018 Dollars/hr
    DollarYearAdjust = 1 + 0.02 # Adjust to 2019 Dollar by Applying Inflation Rate (CPI)
    Annual_Time_BenefitDollar = DollarYearAdjust * TruckValueOfTime * Annual_Time_BenefitMinutes/60
    result = int(round(Annual_Time_BenefitDollar,rounding2))
    truck_table.at[index,"Annual Truck Time Savings [2019$]"] = result
    
# truck_table = truck_table.drop(columns=["Alternative Databank","BAU Databank"])
truck_table = truck_table[["Alternative","Horizon","Annual Truck Time Saving Hours"]]
truck_table.to_csv("D3.csv",index=False)  
truck_table

,Alternative,Horizon,Annual Truck Time Saving Hours
0,Alt_1A,2035,44320


In [24]:
# summarize csv files into one excel file
csv_list = ["INFO"] + pd.read_csv("INFO.csv")["Item"].to_list()

writer = pd.ExcelWriter('UBCx_MAE.xlsx', engine='xlsxwriter')
for sheetname in csv_list:
    try:
        pd.read_csv(sheetname+".csv").to_excel(writer, sheet_name=sheetname, index=False)
    except:
        print(sheetname+".csv")

writer.save()